In [ ]:
#Extrator de Dados
import numpy as np
import time
import requests
import pandas as pd
import logging
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

# Configuração de Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Definição do intervalo de datas para coleta de dados
start_date = datetime(2025, 1, 17)
end_date = datetime(2025, 1, 29)

# Detalhes da requisição da API
url_events = "https://api.b365api.com/v3/events/ended"
url_odds = "https://api.b365api.com/v2/event/odds"
token = "183604-pWN7flhoAsWGu8"  # Substitua pelo seu token real

# Função genérica para requisições com retries
def make_request_with_retry(url, params, retries=5, backoff_factor=2, timeout=10):
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()  # Levanta exceção se status_code >= 400
            return response
        except requests.RequestException as e:
            wait_time = backoff_factor ** attempt  # Exponential backoff
            logging.warning(f"Tentativa {attempt} falhou: {e}. Retentando em {wait_time} segundos.")
            time.sleep(wait_time)
            if attempt == retries:
                logging.error(f"Todas as {retries} tentativas falharam para URL: {url}")
                raise e

# Função para buscar eventos em uma data específica
def fetch_events_for_date(date):
    formatted_date = date.strftime('%Y%m%d')
    params = {
        "token": token,
        "sport_id": "1",
        "league_id": "22614",
        "day": formatted_date,
        "page": 1
    }
    events = []
    while True:
        try:
            response = make_request_with_retry(url_events, params)
            data = response.json()
            events.extend(data['results'])
            if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                params['page'] += 1
            else:
                break
        except Exception as e:
            logging.error(f"Erro ao buscar eventos para a data {formatted_date}: {e}")
            break
    return events

# Função para pegar os dados da API de odds
def fetch_odds_for_event(event_id):
    params = {
        'event_id': event_id,
        'token': token
    }
    try:
        response = make_request_with_retry(url_odds, params)
        data = response.json()
        if 'success' in data and data['success'] == 1:
            results = data.get('results', {})
            odds = results.get('odds', {})

            markets = ['1_1', '1_2', '1_3']
            filtered_odds = {}

            for market in markets:
                market_odds = odds.get(market, [])
                selected_odd = None
                min_add_time = float('inf')

                for odd in market_odds:
                    add_time = int(odd.get('add_time'))
                    ss = odd.get('ss')

                    if ss == '0-0' and add_time < min_add_time:
                        min_add_time = add_time
                        selected_odd = odd

                if not selected_odd:
                    max_add_time = float('-inf')
                    for odd in market_odds:
                        add_time = int(odd.get('add_time'))
                        ss = odd.get('ss')

                        if ss is None and add_time > max_add_time:
                            max_add_time = add_time
                            selected_odd = odd

                if selected_odd:
                    if market == '1_3':
                        filtered_odds[market] = {
                            'over_od': selected_odd.get('over_od'),
                            'under_od': selected_odd.get('under_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                    else:
                        filtered_odds[market] = {
                            'home_od': selected_odd.get('home_od'),
                            'draw_od': selected_odd.get('draw_od'),
                            'away_od': selected_odd.get('away_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                else:
                    filtered_odds[market] = None
            return filtered_odds
    except Exception as e:
        logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
    return None

# Coleta de eventos para cada data no intervalo
all_events = []
for single_date in pd.date_range(start_date, end_date):
    events = fetch_events_for_date(single_date)
    all_events.extend(events)
    logging.info(f"Fetched {len(events)} events for {single_date.strftime('%Y-%m-%d')}")

# Criando o DataFrame com os eventos coletados
df_events = pd.DataFrame([{
    'event_id': event['id'],
    'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),
    'time_fora': event['away']['name'].split('(')[0].strip(),
    'jogador_fora': event['away']['name'].split('(')[1].split(')')[0] if '(' in event['away']['name'] else '',
    'gols_fora': event.get('scores', {}).get('2', {}).get('away', 'N/A'),
    'time_casa': event['home']['name'].split('(')[0].strip(),
    'jogador_casa': event['home']['name'].split('(')[1].split(')')[0] if '(' in event['home']['name'] else '',
    'gols_casa': event.get('scores', {}).get('2', {}).get('home', 'N/A'),
    #'corners_fora': event.get('stats', {}).get('corners', [np.nan, np.nan])[1],
    #'corners_casa': event.get('stats', {}).get('corners', [np.nan, np.nan])[0],
} for event in all_events])

# Conversão de 'gols_casa', 'gols_fora', 'corners_casa' e 'corners_fora' para numérico e remoção de linhas com NaN
df_events['gols_casa'] = pd.to_numeric(df_events['gols_casa'].replace('N/A', np.nan), errors='coerce')
df_events['gols_fora'] = pd.to_numeric(df_events['gols_fora'].replace('N/A', np.nan), errors='coerce')
#df_events['corners_casa'] = pd.to_numeric(df_events['corners_casa'].replace('N/A', np.nan), errors='coerce')
#df_events['corners_fora'] = pd.to_numeric(df_events['corners_fora'].replace('N/A', np.nan), errors='coerce')

# Adicionando a coluna 'gols_totais' pela soma de 'gols_casa' e 'gols_fora'
df_events['gols_totais'] = df_events['gols_casa'] + df_events['gols_fora']

# Ordenação dos eventos por data
df_events.sort_values(by='date', inplace=True)

# Função para processar cada linha para odds
def process_event_odds(index, row):
    event_id = row['event_id']
    logging.info(f"Fetching odds for event_id: {event_id}")
    try:
        odds_data = fetch_odds_for_event(event_id)
        resultado = {}

        if odds_data:
            for market, odds in odds_data.items():
                if odds:
                    if market == '1_3':
                        resultado[f'{market}_over_od'] = odds['over_od']
                        resultado[f'{market}_under_od'] = odds['under_od']
                        resultado[f'{market}_handicap'] = odds['handicap']
                    #else:
                        #resultado[f'{market}_home_od'] = odds['home_od']
                        #resultado[f'{market}_draw_od'] = odds['draw_od']
                        #resultado[f'{market}_away_od'] = odds['away_od']
                        #resultado[f'{market}_handicap'] = odds['handicap']
                        
                    #resultado[f'{market}_add_time'] = odds['add_time']

        return index, resultado
    except Exception as e:
        logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
        return index, {}

# Adicionando odds ao DataFrame de eventos
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_event_odds, index, row) for index, row in df_events.iterrows()]
    for future in as_completed(futures):
        index, odds_result = future.result()
        for col, value in odds_result.items():
            df_events.loc[index, col] = value
        logging.info(f"Odds added for event_id at index {index}")

# Exibição dos primeiros registros para verificação
display(df_events.head())

# Salvar o DataFrame atualizado com odds
df_events.to_csv('dados_8min.csv', index=False)
df_events.to_excel('dados_8min.xlsx', index=False)
logging.info("Processamento concluído e dados salvos.")


In [ ]:
# Extrator de Dados
import numpy as np
import time
import requests
import pandas as pd
import logging
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

# Configuração de Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Definição do intervalo de datas para coleta de dados
start_date = datetime(2025, 1, 17)
end_date = datetime(2025, 2, 4)

# Detalhes da requisição da API
url_events = "https://api.b365api.com/v3/events/ended"
url_odds = "https://api.b365api.com/v2/event/odds"
token = "183604-pWN7flhoAsWGu8"  # Substitua pelo seu token real
league_ids = [22614, 23114, 37298, 38439]  # Lista de ligas a serem coletadas

# Função genérica para requisições com retries
def make_request_with_retry(url, params, retries=5, backoff_factor=2, timeout=10):
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()  # Levanta exceção se status_code >= 400
            return response
        except requests.RequestException as e:
            wait_time = backoff_factor ** attempt  # Exponential backoff
            logging.warning(f"Tentativa {attempt} falhou: {e}. Retentando em {wait_time} segundos.")
            time.sleep(wait_time)
            if attempt == retries:
                logging.error(f"Todas as {retries} tentativas falharam para URL: {url}")
                raise e

# Função para buscar eventos em uma data e liga específicas
def fetch_events_for_date(date, league_id):
    formatted_date = date.strftime('%Y%m%d')
    params = {
        "token": token,
        "sport_id": "1",
        "league_id": str(league_id),
        "day": formatted_date,
        "page": 1
    }
    events = []
    while True:
        try:
            response = make_request_with_retry(url_events, params)
            data = response.json()
            current_events = data['results']
            # Adiciona league_id a cada evento
            for event in current_events:
                event['league_id'] = league_id
            events.extend(current_events)
            if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                params['page'] += 1
            else:
                break
        except Exception as e:
            logging.error(f"Erro ao buscar eventos para a data {formatted_date} e liga {league_id}: {e}")
            break
    return events

# Função para pegar os dados da API de odds
def fetch_odds_for_event(event_id):
    params = {
        'event_id': event_id,
        'token': token
    }
    try:
        response = make_request_with_retry(url_odds, params)
        data = response.json()
        if 'success' in data and data['success'] == 1:
            results = data.get('results', {})
            odds = results.get('odds', {})

            markets = ['1_1', '1_2', '1_3']
            filtered_odds = {}

            for market in markets:
                market_odds = odds.get(market, [])
                selected_odd = None
                min_add_time = float('inf')

                for odd in market_odds:
                    add_time = int(odd.get('add_time'))
                    ss = odd.get('ss')

                    if ss == '0-0' and add_time < min_add_time:
                        min_add_time = add_time
                        selected_odd = odd

                if not selected_odd:
                    max_add_time = float('-inf')
                    for odd in market_odds:
                        add_time = int(odd.get('add_time'))
                        ss = odd.get('ss')

                        if ss is None and add_time > max_add_time:
                            max_add_time = add_time
                            selected_odd = odd

                if selected_odd:
                    if market == '1_3':
                        filtered_odds[market] = {
                            'over_od': selected_odd.get('over_od'),
                            'under_od': selected_odd.get('under_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                    else:
                        filtered_odds[market] = {
                            'home_od': selected_odd.get('home_od'),
                            'draw_od': selected_odd.get('draw_od'),
                            'away_od': selected_odd.get('away_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                else:
                    filtered_odds[market] = None
            return filtered_odds
    except Exception as e:
        logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
    return None

# Coleta de eventos para cada data e liga no intervalo
all_events = []
for single_date in pd.date_range(start_date, end_date):
    for league_id in league_ids:
        events = fetch_events_for_date(single_date, league_id)
        all_events.extend(events)
        logging.info(f"Coletados {len(events)} eventos para liga {league_id} em {single_date.strftime('%Y-%m-%d')}")

# Criando o DataFrame com os eventos coletados
df_events = pd.DataFrame([{
    'event_id': event['id'],
    'league': event['league_id'],  # Nova coluna adicionada
    'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),
    'time_fora': event['away']['name'].split('(')[0].strip(),
    'jogador_fora': event['away']['name'].split('(')[1].split(')')[0] if '(' in event['away']['name'] else '',
    'gols_fora': int(event.get('scores', {}).get('2', {}).get('away', 0)),  # Convertendo para inteiro
    'time_casa': event['home']['name'].split('(')[0].strip(),
    'jogador_casa': event['home']['name'].split('(')[1].split(')')[0] if '(' in event['home']['name'] else '',
    'gols_casa': int(event.get('scores', {}).get('2', {}).get('home', 0)),  # Convertendo para inteiro
} for event in all_events])

# Conversão de 'gols_casa', 'gols_fora', 'corners_casa' e 'corners_fora' para numérico e remoção de linhas com NaN
df_events['gols_casa'] = pd.to_numeric(df_events['gols_casa'].replace('N/A', np.nan), errors='coerce')
df_events['gols_fora'] = pd.to_numeric(df_events['gols_fora'].replace('N/A', np.nan), errors='coerce')

df_events['gols_totais'] = df_events['gols_casa'] + df_events['gols_fora']

# Função para processar cada linha para odds
def process_event_odds(index, row):
    event_id = row['event_id']
    logging.info(f"Buscando odds para event_id: {event_id}")
    try:
        odds_data = fetch_odds_for_event(event_id)
        resultado = {}

        if odds_data:
            for market, odds in odds_data.items():
                if odds:
                    if market == '1_3':
                        resultado[f'{market}_over_od'] = odds['over_od']
                        resultado[f'{market}_under_od'] = odds['under_od']
                        resultado[f'{market}_handicap'] = odds['handicap']
                    else:
                        resultado[f'{market}_home_od'] = odds['home_od']
                        resultado[f'{market}_draw_od'] = odds['draw_od']
                        resultado[f'{market}_away_od'] = odds['away_od']
                        resultado[f'{market}_handicap'] = odds['handicap']
                        
                    resultado[f'{market}_add_time'] = odds['add_time']

        return index, resultado
    except Exception as e:
        logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
        return index, {}

# Adicionando odds ao DataFrame de eventos
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_event_odds, index, row) for index, row in df_events.iterrows()]
    for future in as_completed(futures):
        index, odds_result = future.result()
        for col, value in odds_result.items():
            df_events.loc[index, col] = value
        logging.info(f"Odds adicionadas para event_id no índice {index}")

# Exibição dos primeiros registros para verificação
display(df_events.head())

# Salvamento dos dados
df_events.to_csv('dados_totais.csv', index=False)
df_events.to_excel('dados_totais.xlsx', index=False)
logging.info("Processamento concluído e dados salvos.")

2025-02-05 10:42:14,375 - INFO - Coletados 378 eventos para liga 22614 em 2025-01-17
2025-02-05 10:42:19,297 - INFO - Coletados 256 eventos para liga 23114 em 2025-01-17
2025-02-05 10:42:24,766 - INFO - Coletados 161 eventos para liga 37298 em 2025-01-17
2025-02-05 10:42:34,025 - INFO - Coletados 280 eventos para liga 38439 em 2025-01-17
2025-02-05 10:42:38,936 - INFO - Coletados 268 eventos para liga 22614 em 2025-01-18
2025-02-05 10:42:46,409 - INFO - Coletados 304 eventos para liga 23114 em 2025-01-18
2025-02-05 10:42:49,592 - INFO - Coletados 183 eventos para liga 37298 em 2025-01-18
2025-02-05 10:42:54,670 - INFO - Coletados 200 eventos para liga 38439 em 2025-01-18
2025-02-05 10:42:58,963 - INFO - Coletados 220 eventos para liga 22614 em 2025-01-19
2025-02-05 10:43:05,828 - INFO - Coletados 276 eventos para liga 23114 em 2025-01-19
2025-02-05 10:43:08,081 - INFO - Coletados 149 eventos para liga 37298 em 2025-01-19
2025-02-05 10:43:13,124 - INFO - Coletados 200 eventos para liga 